Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [89]:
from random import choices, random, randint
from queue import PriorityQueue
import numpy as np
%pip install sortedcontainers
from sortedcontainers import SortedList

import lab9_lib

Note: you may need to restart the kernel to use updated packages.


# Utils 

In [90]:
def print_ind(ind):
    return f"{''.join(str(g) for g in ind)}"

def _mutate(a, prob: float):
    if random()<prob:
        return 1-a
    else:
        return a
    
def _mutation_mask(a,b):
    if a!=b:
        return b
    else:
        return -1
    
def _reapply_mutation(a,b):
    if a==-1:
        return b
    else:
        return a
    
def _crossover(a, b):
    if random()>0.5:
        return a
    else:
        return b
    
def gen_population(start_population:int, fitness:callable):
    population=SortedList(key=lambda e: -e[0])
    for i in range(start_population):
        ind=choices([0, 1], k=1000)
        fit=fitness(ind)
        population.add([fit,ind])
    return population

def print_population(population, verbose: bool):
    if not verbose:
        return
    for i in population:
        print(f"{i[0]:.2%} {print_ind(i[1])}")
    
gene_prob=[0 for _ in range(1000)]


mutate=np.vectorize(_mutate)
mutation_mask=np.vectorize(_mutation_mask)
crossover=np.vectorize(_crossover)

# Absolute Perfect Solution

In [91]:
def absolute_perfect_solution():
    return [1 for i in range(0,50)]

# Just Mutation

Basically kills some individuals, mutate some others (and puts back both the original and the new one) \
With: \
Mutation(100, 0.1,0.5,0.1,fitness, verbose=False)
- Problem 2: 96-100%
- Problem 5: 75-80%
- Problem 10: 70-80%

In [92]:
class Mutation():
    def __init__(self, start_population: int, kill_perc: float, mut_prob: float, gene_mut_prob:float, fitness: callable , with_removal:bool=False, verbose:bool=False, max_population:int=1000):
        self.population=gen_population(start_population, fitness)
        self.mut_prob=mut_prob
        self.gene_mut_prob=gene_mut_prob
        self.kill_perc=kill_perc
        self.epochs=0
        self.verbose=verbose
        self.fitness=fitness
        self.with_removal=with_removal
        self.max_population=max_population
        
        print_population(self.population, self.verbose)

    def step(self):
        global mutate
        
        self.epochs+=1

        ### KILL ###
        kill_population(self.population, self.kill_perc, self.max_population)
        
        ### MUTATE ###
        mutate_population(self.population, self.mut_prob, self.gene_mut_prob, self.fitness, self.with_removal)
                    #print(f"After: {i[0]:.2%} {print_ind(i[1])}")
        print("Epoch: ", self.epochs," individuals: ", len(self.population), " best fitness: ", self.best[0])
        #print(print_ind(self.best[1]))
        print_population(self.population, self.verbose)
    
    @property
    def best(self):
        return self.population[0]
    
def kill_population(population:list, kill_perc:float, max_pop=None):
    to_remove=int(len(population)*kill_perc)

    for _ in range(to_remove):
        population.pop()
    if max_pop!=None:
        while(len(population)>max_pop):
            population.pop()
            
        
    
def mutate_population(population,  population_sub:list, mut_prob: float, gene_mut_prob:float, fitness: callable , with_removal:bool=False):
    global gene_prob
    mutations=0
    for i in population_sub:
        #prob=(1 - (i[0]/100)) * mut_prob 
        prob_ind=[0 for _ in range(1000)]
        prob=mut_prob
        if(random()<prob):
            t=mutate(i[1], gene_mut_prob*(1-i[0]))
            t_fit=fitness(t)
            if(with_removal):
                population.remove(i)
            population.add([t_fit,t.tolist()])
            mutations+=1
                


# Crossover

With random crossover it seems mutation gives better solutions \

With this setting in can reach 100% of fitness in a range o
cross=Crossover(50, 0.1, 5, .6, .5, fitness, verbose=False, with_removal=True)
for _ in range(10000):
    cross.step()

In [93]:
class Crossover():
    def __init__(self, start_population: int, kill_perc: float, offspring: int, mut_prob: float, gene_mut_prob:float, fitness: callable ,max_population:int=1000, with_removal:bool=False, verbose:bool=False):
        self.population=gen_population(start_population, fitness)
        self.kill_perc=kill_perc
        self.offspring=offspring
        self.mut_prob=mut_prob
        self.gene_mut_prob=gene_mut_prob
        self.fitness=fitness
        self.with_removal=with_removal
        self.verbose=verbose
        self.max_population=max_population
        self.epochs=0

        print_population(self.population, self.verbose)

    def step(self, elite_threshold=0.2, losers_pool=None, max_loser=600):
        global mutate, crossover, loser_population
        
        self.epochs+=1

        
        elite=int(elite_threshold*len(self.population))

        n_couples=int(0.5*elite*0.5)

        

        for _ in range(n_couples):
            crossover_population_by_difference(self.population, self.population, self.offspring, self.fitness)

        
        mutate_population(self.population, self.population[elite+1:], self.mut_prob, self.gene_mut_prob,self.fitness, self.with_removal)
        
        if losers_pool is None:
            kill_population(self.population, self.kill_perc,self.max_population)
        else:
            loser_population(self.population, losers_pool, self.kill_perc,self.max_population, max_loser)

        print("Epoch: ", self.epochs," individuals: ", len(self.population), " best fitness: ", self.best[0], " ",self.population[-1][0])
        #print(print_ind(self.best[1]))
        print_population(self.population, self.verbose)
    @property
    def best(self):
        return self.population[0]
    
    
    
def inject_new_individuals(population:SortedList, fitness, n=5):
    for _ in range(n):
        ind=choices([0, 1], k=1000)
        fit=fitness(ind)
        population.add([fit,ind])
    
def crossover_population(population_sub, population,offspring, fitness):
    #selected=[0 for _ in population]
    #print("Selected len: ", len(selected), " population: ", len(population))

    p1=0
    p2=0
    #while(True):
    p1=randint(0,len(population_sub)-1)
    p2=randint(0,len(population_sub)-1)
        # print("P1: ",p1," P2: ",p2, len(population))
        # if p2!=p1 and (selected[p1]==0) and (selected[p2]==0):
        #     break
    #selected[p1]=1
    #selected[p2]=1

    parent1=population_sub[p1]
    parent2=population_sub[p2]

    for _ in range(offspring):
        ind=crossover(parent1[1],parent2[1]).tolist()
        fit=fitness(ind)
        if ind!= parent1[1] and ind != parent2[1]:
            population.add([fit,ind])

def crossover_population_by_difference(population_sub, population,offspring, fitness):
    #selected=[0 for _ in population]
    #print("Selected len: ", len(selected), " population: ", len(population))

    p1=0
    p2=0
    #while(True):
    p1=randint(0,len(population_sub)-1)
    p2=0
        # print("P1: ",p1," P2: ",p2, len(population))
        # if p2!=p1 and (selected[p1]==0) and (selected[p2]==0):
        #     break
    #selected[p1]=1
    #selected[p2]=1
    parent1=population_sub[p1]
    parent2=population_sub[p2]
    maxent=0
    for i in range(len(population_sub)):
        temp=population_sub[i]
        e=entropy(parent1[1],temp[1])
        
        if e>maxent:
            maxent=e
            parent2=temp

    #print(f"{print_ind(temp[1])}, {e}")

    for _ in range(offspring):
        ind=crossover(parent1[1],parent2[1]).tolist()
        if ind!= parent1[1] and ind != parent2[1]:
            fit=fitness(ind)
            population.add([fit,ind])

def entropy(ind1, ind2):
    return sum([1 for i in range(len(ind1)) if ind1[i]!=ind2[1]])
        

In [94]:
def remove_similar(population:SortedList):
    individuals=[]
    for i in population:
        if i in individuals:
            population.remove(i)
        else:
            individuals.append(i)

# Phenotype Study

# Testing Solution

Crossover + Islands + Mass Extinctions: 
start ind=50 
kill_perc=0.5 
offsprings=15
mutation_prob= 0.8
gene_mutation_prob = .5
with_removal=False
max_population=200

Problem 2: 100% in 1.3M fitness calls
Problem 5: 100% in 1.5M fitness calls


In [96]:
fitness = lab9_lib.make_problem(10)

#mut=Mutation(100, 0,0.1,.2,fitness, verbose=False, max_population=500)
#for i in range(10000):
#    mut.step()
#    if ((i+1)%40) == 0:
#        inject_new_individuals(cross.population,fitness,30)
# best=mut.best
# print(f"{best[0]:.2%} {print_ind(best[1])}")

losers_pool=SortedList()
cross1=Crossover(50, 0.5, 15, .8, .5, fitness, verbose=False, with_removal=False,max_population=200)
cross2=Crossover(50, 0.5, 15, .8, .5, fitness, verbose=False, with_removal=False,max_population=200)


for i in range(10000):
    cross1.step(.2)
    cross2.step(.2)
    
    # if i%40==0:
    #    if len(losers_pool)>20:
    #        cross1.population.update(losers_pool[:])
    #        cross2.population.update(losers_pool[:])
    #    kill_population(losers_pool, 1,100)
    #    inject_new_individuals(cross2.population,fitness, 30)
    #    inject_new_individuals(cross1.population,fitness, 30)
    #    print(len(losers_pool))
    remove_similar(cross1.population)
    remove_similar(cross2.population)

    

    #mutate_population(losers_pool,losers_pool, 0.8,.5,fitness)
    #kill_population(losers_pool, 0,100)
    #remove_similar(losers_pool)

    # if len(cross1.population)>5 and len(cross2.population)>5:
    #     cross1.population.update(cross2.population[:5])
    #     cross2.population.update(cross1.population[:5])

    if (i+1 )% 40==0:
        kill_population(cross1.population, 0.9,200)
        kill_population(cross2.population, 0.9,200)
        inject_new_individuals(cross2.population,fitness, 30)
        inject_new_individuals(cross1.population,fitness, 30)
        cross1.population.update(cross2.population[:5])
        cross2.population.update(cross1.population[:5])
    
    print(f"\nCalls {fitness.calls}")
    if cross1.best[0]==1.0 or cross2.best[0]==1.0:
        break


best1=cross1.best
best2=cross2.best

print(f"{best1[0]:.2%} {print_ind(best1[1])}")
print(f"{best2[0]:.2%} {print_ind(best2[1])}")

print(f"\nCalls {fitness.calls}")

Epoch:  1  individuals:  68  best fitness:  0.194691142   0.051791113691
Epoch:  1  individuals:  68  best fitness:  0.1621367027   0.051922466688

Calls 272
Epoch:  2  individuals:  100  best fitness:  0.198468009   0.052590124896
Epoch:  2  individuals:  93  best fitness:  0.206226041   0.053122468129

Calls 520
Epoch:  3  individuals:  148  best fitness:  0.202335798   0.053356790006
Epoch:  3  individuals:  130  best fitness:  0.21805568399999997   0.053714689120000006

Calls 881


Epoch:  4  individuals:  200  best fitness:  0.214566906   0.054344566829999996
Epoch:  4  individuals:  187  best fitness:  0.21805568399999997   0.054445792507999996

Calls 1404
Epoch:  5  individuals:  200  best fitness:  0.221913566   0.057925577898
Epoch:  5  individuals:  200  best fitness:  0.21805568399999997   0.056023448005999996

Calls 2168
Epoch:  6  individuals:  200  best fitness:  0.221913566   0.10024455829
Epoch:  6  individuals:  200  best fitness:  0.221813806   0.10022789017

Calls 2940
Epoch:  7  individuals:  200  best fitness:  0.221913566   0.10400014007
Epoch:  7  individuals:  200  best fitness:  0.221813806   0.10411346711

Calls 3719
Epoch:  8  individuals:  200  best fitness:  0.221913566   0.10466667907
Epoch:  8  individuals:  200  best fitness:  0.221813806   0.10600033461

Calls 4520
Epoch:  9  individuals:  200  best fitness:  0.221913566   0.10602456796
Epoch:  9  individuals:  200  best fitness:  0.221813806   0.10779013485

Calls 5308
Epoch:  10  in